In [ ]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

In [ ]:
plates = [
# "111_20240131",
# "179_20240219",
# "199_20240130",
"393_20231129",
# "401_20231123",
# "404_20231127",
# "407_20240102"
]
directory_targ = directory_project
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == "393_20231129"]
folders = folders.sort_values(by="datetime")
folders = folders.iloc[72:73]

exp = Experiment(directory_targ)
exp.load(folders)
exp.dates.sort()
for t in range(len(folders)):
    exp.load_tile_information(t)

In [ ]:
load_graphs(exp, directory_targ)
exp.save_location = ""

load_study_zone(exp)

In [ ]:
import networkx as nx

G = exp.nx_graph[t]
components = nx.connected_components(G)

# Find the largest connected component
largest_component = max(components, key=len)

# Create a new graph representing the largest connected component
largest_component_graph = G.subgraph(largest_component)
exp.nx_graph[t] = largest_component_graph

In [ ]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)
nodes_source = [
    node
    for node in nodes
    if not is_in_study_zone(node, t, 1000, 150)[1]
    and is_in_study_zone(node, t, 1000, 150)[0]
]
nodes_sink = [
    node
    for node in nodes
    if is_in_study_zone(node, t, 1000, 150)[1] and node.degree(t) == 1
]

In [ ]:
nodes_source_label = [
    562057,
    562065,
    511904,
    512822,
    512830,
    529474,
    228,
    229,
    564159,
    230,
    562095,
    527119,
]
nodes_source2 = [Node(label, exp) for label in nodes_source_label]

In [ ]:
ax = plot_full(exp, t, nodes=nodes_source)
# ax.scatter(
#     spore_datatable["y"] / 5, spore_datatable["x"] / 5, s=20, color="blue", alpha=0.4
# )

In [ ]:
f = lambda edge: np.log(
    edge.width(t) * edge.length_um(t) * edge.end.degree(t) * edge.begin.degree(t)
)
edges = get_all_edges(exp, t)
values = [f(edge) for edge in edges]
weights = [edge.length_um(t) for edge in edges]


In [ ]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)
# nodes_source = [
#     node
#     for node in nodes
#     if not is_in_study_zone(node, t, 1000, 150)[1]
#     and is_in_study_zone(node, t, 1000, 150)[0]
# ]
nodes_source = nodes_source
nodes_sink = [
    node
    for node in nodes
    if is_in_study_zone(node, t, 1000, 150)[1] and node.degree(t) == 1
]
weights = {(edge.begin.label, edge.end.label): edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t], weights, "length")
weights = {(edge.begin.label, edge.end.label): 1 / edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t], weights, "1/length")
t = 0
G = exp.nx_graph[t]
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
len_connected = [len(nx_graph.nodes) for nx_graph in S]
final_current_flow_betweeness = {}
final_betweeness = {}

for g in S:
    source = [node.label for node in nodes_source if node.label in g]
    sink = [node.label for node in nodes_sink if node.label in g]
    # current_flow_betweeness = nx.edge_current_flow_betweenness_centrality_subset(
    #     g, source, sink, weight="1/length"
    # )
    # betweeness = nx.edge_current_flow_betweenness_centrality_subset(
    #     g, sink, source, weight="length"
    # )

    betweeness = nx.edge_betweenness_centrality_subset(
        g, source, sink, normalized=True, weight="length"
    )
    # for edge in current_flow_betweeness.keys():
    #     final_current_flow_betweeness[edge] = current_flow_betweeness[edge]
    for edge in betweeness.keys():
        final_betweeness[edge] = betweeness[edge]

for edge in exp.nx_graph[t].edges:
    # if (
    #     edge not in final_current_flow_betweeness.keys()
    #     and (edge[1], edge[0]) not in final_current_flow_betweeness.keys()
    # ):
    #     final_current_flow_betweeness[edge] = 0
    if (
        edge not in final_betweeness.keys()
        and (edge[1], edge[0]) not in final_betweeness.keys()
    ):
        final_betweeness[edge] = 0
nx.set_edge_attributes(exp.nx_graph[t], final_betweeness, "betweenness")

In [ ]:
vmax = 12
vmin = 2
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    cmap=cm.get_cmap("cool", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
)

In [ ]:
plt.close("all")
vmax = -2.2
vmin = -4.5
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.betweeness(t)),
    cmap=cm.get_cmap("autumn", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize = (12,7)
)

In [ ]:
plt.close("all")
vmax = -2.2
vmin = -4.5
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.current_flow_betweeness(t)),
    cmap=cm.get_cmap("cool", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
)
path = r"/scratch-shared/amftrack/transport/full_picture/"
file_name = f"betweeness_random.png"
save_path = os.path.join(path, file_name)
plt.savefig(save_path, dpi=dpi)
file_name = f"betweeness_random.csv"
save_path = os.path.join(path, file_name)
plt.savefig(save_path)
file_name = f"betweeness_random.eps"
save_path = os.path.join(path, file_name)
plt.savefig(save_path)

In [ ]:
vmax = 0
vmin = 10
dpi = 400

# ax = plot_edge_color_value(exp,t,lambda edge : 5,cmap = cm.get_cmap("Reds", 100),v_min =vmin,v_max = vmax,plot_cmap=True,show_background=True,dilation =10,figsize = (12,8))
ax = plot_full(exp, t)
ax.scatter(
    spore_datatable["y"] / 5, spore_datatable["x"] / 5, s=20, color="blue", alpha=0.4
)
path = r"/scratch-shared/amftrack/transport/full_picture/"
file_name = f"spores.png"
save_path = os.path.join(path, file_name)
# plt.savefig(save_path, dpi=dpi)

In [ ]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)

In [ ]:
nodes_source = [
    node
    for node in nodes
    if not is_in_study_zone(node, t, 1000, 150)[1]
    and is_in_study_zone(node, t, 1000, 150)[0]
]

In [ ]:
nodes_sink = [
    node
    for node in nodes
    if is_in_study_zone(node, t, 1000, 150)[1] and node.degree(t) == 1
]
# nodes_sink = [node for node in nodes if is_in_study_zone(node,t,1000,150)[1] ]

In [ ]:
len(nodes_source), len(nodes_sink)

In [ ]:
weights = {(edge.begin.label, edge.end.label): edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t], weights, "length")
weights = {(edge.begin.label, edge.end.label): 1 / edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t], weights, "1/length")

In [ ]:
t = 0
G = exp.nx_graph[t]
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
len_connected = [len(nx_graph.nodes) for nx_graph in S]
final_current_flow_betweeness = {}
final_betweeness = {}

for g in S:
    source = [node.label for node in nodes_source if node.label in g]
    sink = [node.label for node in nodes_sink if node.label in g]
    current_flow_betweeness = nx.edge_current_flow_betweenness_centrality_subset(
        g, source, sink, weight="1/length"
    )
    betweeness = nx.edge_current_flow_betweenness_centrality_subset(
        g, sink, source, weight="length"
    )

    # betweeness = nx.edge_betweenness_centrality_subset(g,source,sink,normalized=True,weight = "length")
    for edge in current_flow_betweeness.keys():
        final_current_flow_betweeness[edge] = current_flow_betweeness[edge]
    for edge in betweeness.keys():
        final_betweeness[edge] = betweeness[edge]

for edge in exp.nx_graph[t].edges:
    if (
        edge not in final_current_flow_betweeness.keys()
        and (edge[1], edge[0]) not in final_current_flow_betweeness.keys()
    ):
        final_current_flow_betweeness[edge] = 0
    if (
        edge not in final_betweeness.keys()
        and (edge[1], edge[0]) not in final_betweeness.keys()
    ):
        final_betweeness[edge] = 0
nx.set_edge_attributes(
    exp.nx_graph[t], final_current_flow_betweeness, "current_flow_betweenness"
)
nx.set_edge_attributes(exp.nx_graph[t], final_betweeness, "betweenness")
betweenessses = list(final_current_flow_betweeness.values())
np.mean(betweenessses), np.max(betweenessses)

In [ ]:
betweenessses = list(final_current_flow_betweeness.values())
betweenessses2 = list(final_betweeness.values())
fig, ax = plt.subplots()
h = ax.hist(np.log10([bet for bet in betweenessses if bet > 1e-4]), 20)

In [ ]:
final_current_flow_betweeness[(527418, 512522)], final_betweeness[(527418, 512522)]

In [ ]:
final_current_flow_betweeness[(527418, 512522)], final_betweeness[(527418, 512522)]

In [ ]:
plt.close("all")
vmax = -2.8
vmin = -4.5
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.current_flow_betweeness(t)),
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(16, 10),
)
# path = r"/scratch-shared/amftrack/transport/full_picture/"
# file_name = f'betweeness_random.png'
# save_path = os.path.join(path,file_name)
# plt.savefig(save_path, dpi=dpi)

In [ ]:
vmax = -2.8
vmin = -4.5
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.betweeness(t)),
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(16, 10),
)
# path = r"/scratch-shared/amftrack/transport/full_picture/"
# file_name = f'betweeness_random.png'
# save_path = os.path.join(path,file_name)
# plt.savefig(save_path, dpi=dpi)

In [ ]:
plt.close("all")
vmax = -2.8
vmin = -4.5
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.current_flow_betweeness(t)),
    cmap=cm.get_cmap("cool", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(16, 10),
)
# path = r"/scratch-shared/amftrack/transport/full_picture/"
# file_name = f'betweeness_random.png'
# save_path = os.path.join(path,file_name)
# plt.savefig(save_path, dpi=dpi)

In [ ]:
betweenessses = list(current_flow_betweeness.values())
vmax = np.log10(np.max(betweenessses))
vmax

In [ ]:
betweenessses = [edge.current_flow_betweeness(t) for edge in edges]
betweenessses = [betweeness for betweeness in betweenessses if betweeness > 1e-5]
np.max(betweenessses), np.mean(betweenessses)
fig, ax = plt.subplots()
ax.hist(np.log10(betweenessses))

In [ ]:
plt.close("all")
# betweenessses = list(current_flow_betweeness.values())
vmax = -2.5
vmin = -4.5

plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.current_flow_betweeness(t)),
    cmap=cm.get_cmap("Reds", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    label_colorbar="log random walk edge betweeness centrality",
)

In [ ]:
betweenessses = [edge.betweeness(t) for edge in edges]
betweenessses = [betweeness for betweeness in betweenessses if betweeness > 0]
np.max(betweenessses), np.mean(betweenessses)
fig, ax = plt.subplots()
ax.hist(np.log10(betweenessses))